In [1]:
from src.neo4jdataloader import DataLoader
import pyspark.sql.functions as functions
import gc
import time
import pandas as pd

In [2]:
dataloader = DataLoader()

In [3]:
veiculos = dataloader.load_data('/home/altieris/datascience/data/curitibaurbs/processed/veiculos/')

In [4]:
#f = "year = '2019' and month='03' and day in ('03','04','05','06','07','08','09','10','11','12','13','14','15','16') and dthr is not null"

f = "date between '2019-03-03' and '2019-03-16' "
vehicles_data = veiculos.select('veic', 'lat', 'lon','cod_linha', \
                functions.date_format(functions.unix_timestamp('dthr', 'dd/MM/yyyy HH:mm:ss').cast('timestamp'),"yyyy-MM-dd'T'HH:mm:ss.SSS-0300").alias('event_timestamp'), \
                functions.date_format(functions.unix_timestamp('dthr', 'dd/MM/yyyy HH:mm:ss').cast('timestamp'),"yyyy-MM-dd").alias('date')) \
.filter(f).distinct()

In [42]:
#vehicles_data.groupby('date').count().toPandas()

,date,count
0,2019-03-03,2465032
1,2019-03-11,322388
2,2019-03-07,5067623
3,2019-03-16,3283052
4,2019-03-05,2464148
5,2019-03-14,1249197
6,2019-03-15,1185917
7,2019-03-10,2486403
8,2019-03-09,3347092
9,2019-03-08,5064954


In [5]:
target_path = '/home/altieris/datascience/data/curitibaurbs/neo4j/veiculos/'
vehicles_data.write.mode('overwrite').format("parquet").save(target_path)

In [6]:
veiculos = dataloader.load_data(target_path)

In [12]:
veiculos_df = veiculos.select('veic','cod_linha','date').distinct().toPandas()

In [15]:
#data_processed = pd.read_csv('tracker-data.tsv', sep='\t',names=['veic','cod_linha','date'])

In [10]:
#data_processed.head()

,veic,cod_linha,date
0,BB309,21,2019-03-09
1,BC306,205,2019-03-09
2,BL335,23,2019-03-08
3,EA195,515,2019-03-08
4,GL321,520,2019-03-08


In [11]:
#veiculos_df = veiculos_df[~(veiculos_df.veic.isin(data_processed.veic) & veiculos_df.cod_linha.isin(data_processed.cod_linha) & veiculos_df.date.isin(data_processed.date))]

In [6]:
veiculos_df.head()

,veic,cod_linha,date
0,BB309,021,2019-03-09
1,BC306,205,2019-03-09
2,BL335,023,2019-03-08
3,EA195,515,2019-03-08
4,GL321,520,2019-03-08


In [7]:
data = dict()

for index, row in veiculos_df.iterrows():
    dataloader.create_positions(veiculos,row['veic'],row['cod_linha'],row['date']) 
    dataloader.connect_events(row['veic'],row['cod_linha'],row['date'])
    #dataloader.create_edge_properties(row['veic'],row['cod_linha'],row['date']) 
    
    data['veic'] = row['veic']
    data['cod_linha'] = row['cod_linha']
    data['date'] = row['date']
    
    df = pd.DataFrame(data, index=[0])

    df.to_csv('tracker-data.tsv', header=False, sep='\t',index=False, mode='a')
        
    # Wait for 5 seconds
    time.sleep(10)

KeyboardInterrupt: 